In [4]:
# 加载库文件
import time
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import LeNet5_infernece
import LeNet5_train
import numpy as np

In [5]:
# 每10s加载一次最新模型
EVAL_INTERVAL_SECS = 10 # 加载的实践间隔
BATCH = 5000

In [6]:
# 定义评估函数
def evaluate(mnist):
    with tf.Graph().as_default() as g:
        # 定义输入输出的格式和验证样本集
        x = tf.placeholder(tf.float32, 
                           shape =[BATCH, 
                                   LeNet5_infernece.IMAGE_SIZE,
                                   LeNet5_infernece.IMAGE_SIZE,
                                   LeNet5_infernece.NUM_CHANNELS],
                           name='x-input')
        y_ = tf.placeholder(tf.float32, [BATCH, LeNet5_infernece.OUTPUT_NODE], name='y-input')
        
        
        # 计算前向传播结果、计算准确率、计算精度
        y = LeNet5_infernece.inference(x, False, None)
        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        
        
        # 构建滑动平均类
        # 通过变量重命名的方式加载模型，这样在前向传播的过程中就不需要调用滑动平均类来获取平均值了
        variable_averages = tf.train.ExponentialMovingAverage(LeNet5_train.MOVING_AVERAGE_DECAY)
        variable_to_restore = variable_averages.variables_to_restore()
        saver = tf.train.Saver(variable_to_restore)
        
        
        # 每个EVAL_INTERVAL_SECS秒调用一次计算正确率的过程以检测训练过程中正确率的变化
        while True:
            with tf.Session() as sess:
                # 获取数据
                # 加载全部的验证数据
                xs = mnist.validation.images[0:BATCH]
                ys = mnist.validation.labels[0:BATCH]
        
                # 调整xs的结构
                reshape_xs = np.reshape(xs, (BATCH,
                                             LeNet5_infernece.IMAGE_SIZE,
                                             LeNet5_infernece.IMAGE_SIZE,
                                             LeNet5_infernece.NUM_CHANNELS))
                
                # tf.train.get_checkpoint_state函数会通过checkpoint文件自动找到目录中最新的文件模型
                ckpt = tf.train.get_checkpoint_state(LeNet5_train.MODEL_SAVE_PATH)
                if ckpt and ckpt.model_checkpoint_path:
                    # 加载模型
                    saver.restore(sess, ckpt.model_checkpoint_path)
                    
                    # 通过文件名得到模型保存时迭代的轮数
                    global_step = ckpt.model_checkpoint_path.split('/')[-1].split('-')[-1]
                    accuracy_score = sess.run(accuracy, feed_dict={x:reshape_xs, y_:ys})
                    print("After %s training steps, validation accuracy =%g" % (global_step, accuracy_score))
                else:
                    print("No Checkpoint file found")
                    time.sleep(EVAL_INTERVAL_SECS)
                
                time.sleep(EVAL_INTERVAL_SECS)
            break;
                
# 主程序
def main(argv=None):
    mnist = input_data.read_data_sets('../datasets/MNIST_data', one_hot=True)
    evaluate(mnist)
    
if __name__ == '__main__':
    main()
        

Extracting ../datasets/MNIST_data\train-images-idx3-ubyte.gz
Extracting ../datasets/MNIST_data\train-labels-idx1-ubyte.gz
Extracting ../datasets/MNIST_data\t10k-images-idx3-ubyte.gz
Extracting ../datasets/MNIST_data\t10k-labels-idx1-ubyte.gz


NameError: name 'true' is not defined